In [30]:
import numpy as np
import sympy
import numba as nb

In [6]:
p = np.linspace(0, 10, 201)
delta_p = 0.05

In [69]:
@nb.jit(nopython=True)
def F(p1, p2, p3):
    return np.array([1,0,.5,-1])

@nb.jit(nopython=True)
def create_p(p1, e_max, B):
    num = int((B*e_max - p1)/0.05 + 1)
    return np.linspace(p1, B*e_max, num)

## Integral 4

In [42]:
@nb.jit(nopython=True)
def J_1(p1, p2, p3):
    return 16/15 * p3**3 * (10*(p1+p2)**2 - 15*(p1+p2)*p3 + 6*p3**2)

@nb.jit(nopython=True)
def A_4(p1, p2):
    x = np.linspace(0,p1)
    y = np.zeros((len(x), 4))
    for i in range(len(x)):
        y[i,:] = F(p1, p2, x[i])* J_1(p1, p2, x[i])
    A = np.zeros((1,4))
    for i in range(4):
        A[0,i] = np.trapz(y[:,i], x)
    return A

In [47]:
@nb.jit(nopython=True)
def I4(p1, p, e_max, B):
    x = create_p(p1, e_max, B)
    y = np.zeros((len(x), 4))
    output = np.zeros((1,4))

    for i in range(len(x)):
        y[i,:] = A_4(p1,x[i])

    for k in range(4):
        output[:,k] = np.trapz(y[:,k],x)
    
    return output

In [9]:
p_1, p_2, p_3 = sympy.symbols('p_1, p_2, p_3')

J1 = 16/15 * p_3**3 * (10*(p_1+p_2)**2 - 15*(p_1+p_2)*p_3 + 6*p_3**2)

B_exp = sympy.integrate(J1, (p_3, 0, p_1))

C_exp = sympy.integrate(B_exp, (p_2, p_1, 20))

C = sympy.lambdify((p_1), C_exp)

In [44]:
for i in range(1, 201):
    p1 = p[i]
    A_0 = I4(p1, p, 10, 2)[0]
    B_0 = C(p1)
    diff = np.abs(A_0[0]-B_0) / (np.abs(A_0[0]) + np.abs(B_0) )
    print(diff)

0.00020914163304262324
0.00020851197391427598
0.00020787682448527762
0.0002072517787671029
0.00020662906756755512
0.00020601641856781032
0.0002053984272730608
0.00020477518519442423
0.00020416205115702426
0.00020355136622694702
0.00020295073977542037
0.0002023449897834917
0.00020173420159598103
0.00020113350407214266
0.00020053534666927795
0.0001999472234198103
0.00019935417030935644
0.00019875626688741784
0.00019816841758777683
0.0001975831782016211
0.00019700793051930535
0.00019642792372235161
0.00019584323192467252
0.00019526854095003795
0.00019469651068435134
0.00019413441388171053
0.0001935677078145638
0.0001929964615091782
0.00019243514821634273
0.00019187652952259128
0.00019132777243845248
0.00019077453696366236
0.00019021688736756422
0.0001896690905487644
0.00018912400726460064
0.00018858870213421644
0.00018804903230951932
0.00018750505761254896
0.0001869708449560257
0.00018643935162089488
0.00018591754323551358
0.0001853914683390226
0.00018486118260120166
0.0001843405594270667

In [31]:
%%timeit
Ifour(p[10], p, 10, 2)[0]

38.8 ms ± 359 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
%%timeit
Ifour(p[10], p, 10, 2)[0]

2.72 ms ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Integral 5

In [70]:
@nb.jit(nopython=True)
def J_2(p1,p2):
    return 16/15 * p2**3 * (10*p1**2 + 5*p1*p2 + p2**2)

@nb.jit(nopython=True)
def A_5(p1, p2):
    x = np.linspace(p1,p2)
    y = np.zeros((len(x), 4))
    for i in range(len(x)):
        y[i,:] = F(p1, p2, x[i])* J_2(p2, p1)
    A = np.zeros((1,4))
    for i in range(4):
        A[0,i] = np.trapz(y[:,i], x)
    return A

In [71]:
@nb.jit(nopython=True)
def I5(p1, p, e_max, B):
    x = create_p(p1, e_max, B)
    y = np.zeros((len(x), 4))
    output = np.zeros((1,4))

    for i in range(len(x)):
        y[i,:] = A_5(p1,x[i])

    for k in range(4):
        output[0,k] = np.trapz(y[:,k],x)
    
    return output

In [66]:
p_one, p_two, p_three = sympy.symbols('p_one, p_two, p_three')
J2 = 16/15 * p_one**3 * (10*p_two**2 + 5*p_one*p_two + p_one**2)

B_exp5 = sympy.integrate(J2, (p_three, p_one, p_two))

C_exp5 = sympy.integrate(B_exp5, (p_two, p_one, 20))

C5 = sympy.lambdify((p_one), C_exp5)

In [67]:
for i in range(1, 201):
    p1 = p[i]
    A_0 = I5(p1, p, 10, 2)[0]
    B_0 = C5(p1)
    diff = np.abs(A_0[0]-B_0) / (np.abs(A_0[0]) + np.abs(B_0) )
    print(diff)

3.143337657781512e-06
3.1460158805375153e-06
3.1328663777169808e-06
3.135520774965686e-06
3.138190123877388e-06
3.1568790722759368e-06
3.1596337992583134e-06
3.146290554469504e-06
3.1490223866275785e-06
3.1517705119485675e-06
3.170816593463891e-06
3.1736546342770363e-06
3.1601153508040257e-06
3.1629313573412035e-06
3.165765015307365e-06
3.1851844476315495e-06
3.1881127040050034e-06
3.174374642568634e-06
3.1772816559876346e-06
3.1802077001076156e-06
3.200017357052557e-06
3.203042841696672e-06
3.1891028088615982e-06
3.1921077733718136e-06
3.195133172663117e-06
3.2153506374336467e-06
3.2184804935142346e-06
3.204334827021494e-06
3.2074448189812133e-06
3.2105766788408294e-06
3.2312202988030514e-06
3.234461820929133e-06
3.2201063755572615e-06
3.2233286241934803e-06
3.226574207020415e-06
3.247663155247017e-06
3.2510238131087173e-06
3.236453942792646e-06
3.2397958531857166e-06
3.2431626018505205e-06
3.264716942564232e-06
3.2682044043446964e-06
3.25341494376052e-06
3.2568841209628208e-06
3.2603

In [68]:
%%timeit
I5(p[12], p, 10, 2)

35.5 ms ± 135 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [72]:
%%timeit
I5(p[12], p, 10, 2)

2.78 ms ± 49.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Integral 6

In [89]:
@nb.jit(nopython=True)
def J_3(p1,p2,p3):
    return 16/15 * ( (p1+p2)**5 - 10*(p1+p2)**2*p3**3 + 15*(p1+p2)*p3**4 - 6*p3**5)

@nb.jit(nopython=True)
def A_6(p1, p2):
    x = np.linspace(p2,p1+p2)
    y = np.zeros((len(x), 4))
    for i in range(len(x)):
        y[i,:] = F(p1, p2, x[i])* J_3(p1, p2, x[i])
    A = np.zeros((1,4))
    for i in range(4):
        A[0,i] = np.trapz(y[:,i], x)
    return A

In [90]:
@nb.jit(nopython=True)
def I6(p1, p, e_max, B):
    x = create_p(p1, e_max, B)
    y = np.zeros((len(x), 4))
    output = np.zeros((1,4))

    for i in range(len(x)):
        y[i,:] = A_6(p1,x[i])

    for k in range(4):
        output[0,k] = np.trapz(y[:,k],x)
    
    return output

In [85]:
pone, ptwo, pthree = sympy.symbols('pone, ptwo, pthree')
J3 = 16/15 * ( (pone+ptwo)**5 - 10*(pone+ptwo)**2*pthree**3 + 15*(pone+ptwo)*pthree**4 - 6*pthree**5)

B_exp6 = sympy.integrate(J3, (pthree, ptwo, ptwo+pone))

C_exp6 = sympy.integrate(B_exp6, (ptwo, pone, 20))

C6 = sympy.lambdify((pone), C_exp6)

In [86]:
for i in range(1, 201):
    p1 = p[i]
    A_0 = I6(p1, p, 10, 2)[0]
    B_0 = C6(p1)
    diff = np.abs(A_0[0]-B_0) / (np.abs(A_0[0]) + np.abs(B_0) )
    print(diff)

0.0002091475148628626
0.00020851269251284096
0.0002078769623108616
0.0002072518532386491
0.0002066291106235347
0.0002060164395987412
0.0002053984420404061
0.00020477519226747357
0.00020416205757526292
0.00020355137141601137
0.00020295074366515536
0.00020234499288139448
0.00020173420338112833
0.00020113350570483227
0.00020053534830375775
0.00019994722450782063
0.00019935417129051856
0.0001987562675732877
0.0001981684182452455
0.00019758317886967575
0.00019700793104370366
0.000196427924216094
0.0001958432322536898
0.00019526854125684822
0.00019469651102123459
0.00019413441418295
0.00019356770807690593
0.0001929964616800729
0.00019243514838530849
0.0001918765297056902
0.00019132777258896465
0.00019077453707816121
0.00019021688748436556
0.00018966909065656307
0.00018912400737342293
0.0001885887022176638
0.0001880490323681059
0.00018750505769738254
0.00018697084502545616
0.0001864393517024751
0.00018591754330186602
0.00018539146838970712
0.00018486118265532357
0.00018434055947291644
0.00018

In [88]:
%%timeit
I6(p[50], p, 10, 2)

40.3 ms ± 385 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [91]:
%%timeit
I6(p[50], p, 10, 2)

2.56 ms ± 5.74 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
